# 교차 검증과 그리드 서치

## 검증 세트
* 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다
* 테스트 세트를 사용하지 않고 측정하는 방법은 훈련세트를 다시 나누면 된다. 이 데이터를 검증세트(validation set) 라고 한다
* 훈련세트의 모델의 평가하고, 검증세트로 모델을 평가한다 
 - 테스트 하고 싶은 매개변수를 바꾸면서 가장 좋은 모델을 찾는다
 - 찾은 매개변수를 사용해 훈련세트와 검증세트를 합쳐 전체 훈련데이터에서 모델을 다시 훈련한다
 - 마지막에 테스트 세트에서 최종 점수를 평가한다

In [1]:
import pandas as pd

wine = pd.read_csv('data/wine/wine.csv')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


* 훈련세트 데이터를 훈련세트와 검증세트로 분할한다
* 훈련세트 
 - 독립변수 : sub_input
 - 종속변수 : sub_target
* 검증세트
 - 독립변수 : val_input
 - 종속변수 : val_target

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, sub_target.shape)
print(val_input.shape, val_target.shape)

(4157, 3) (4157,)
(1040, 3) (1040,)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

#훈련세트에 과대적합 되어 있음
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증
* 교차검증(cross validation)은 검증세트를 떼어 내고 평가하는 과정을 여러번 반복한다
* 훈련세트를 세 부분으로 나누어서 교차 검증을 수행하는 것을 3-폴드 교차 검증이라고 한다
 - k-폴드 교차검증(k-Fold cross validation)이라고 한다
* fit_time과 score_time은 모델을 훈련하는 시간과 검증시간을 의미한다
* cross_validate()는 5-폴드 교차 검증을 수행한다

In [7]:
from sklearn.model_selection import cross_validate

dt = DecisionTreeClassifier(random_state=42)
scores = cross_validate(dt, X_train, y_train)
print(scores)

{'fit_time': array([0.00685668, 0.00599384, 0.00500035, 0.00504017, 0.00399947]), 'score_time': array([0.00197697, 0.        , 0.00099993, 0.00096202, 0.00099826]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


* 교차검증의 최종 점수는 test_score에 담긴 5개 점수의 평균하면 알 수 있다
* test_score이지만 검증 폴드의 점수이다

In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


* cross_validate()는 회귀 모델일 경우 KFold를 사용하고
* 분류 모델일 경우 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다

In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, X_train, y_train, cv=StratifiedKFold())
print(scores)
print(np.mean(scores['test_score']))

{'fit_time': array([0.00550675, 0.00610423, 0.0049994 , 0.00500059, 0.00523067]), 'score_time': array([0.        , 0.        , 0.00099993, 0.00100517, 0.00095034]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


* 훈련세트를 섞은 후에 10-폴드 교차 검증을 수행한다

In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, X_train, y_train, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [15]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

print('훈련', dt.score(X_train, y_train))
print('테스트', dt.score(X_test, y_test))

훈련 0.996921300750433
테스트 0.8584615384615385


## 하이퍼파라미터 튜닝
* 모델이 학습하는 파라미터를 모델 파라미터라고 부른다
* 모델이 학습할 수 없어 사용자가 지정해야 하는 파라미터를 하이퍼파라미터 라고 부른다
* GridSearchCV는 하이퍼파라미터와 탐색과 교차검증을 한 번에 수행한다. 
 - cross_validate() 함수를 호출할 필요가 없다

In [16]:
from sklearn.model_selection import GridSearchCV

#max_depth 와 같은 가지치기를 할 때 사용하는 매개변수
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [17]:
model = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(model, params, n_jobs=-1)

In [18]:
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [19]:
dt = gs.best_estimator_
print(dt.score(X_train, y_train))

0.9615162593804117


In [20]:
# 최적의 파라미터
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


* 각 매개변수에서 수행한 교차검증의 평균점수는 cv_results_ 속성의 mean_test_score에 저장되어 있다
* 5번의 교차검증으로 얻은 점수는 아래와 같다

In [21]:
gs.cv_results_

{'mean_fit_time': array([0.00579863, 0.0050004 , 0.00520253, 0.0061985 , 0.00439939]),
 'std_fit_time': array([0.00098245, 0.00063038, 0.00039877, 0.00116625, 0.00048932]),
 'mean_score_time': array([0.0006001 , 0.00079899, 0.00099807, 0.00120001, 0.00080032]),
 'std_score_time': array([4.89979265e-04, 3.99503391e-04, 4.37132516e-06, 4.00161828e-04,
        4.00163647e-04]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.8

In [22]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


* numpy의 argmax()를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다
* 그 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다

In [23]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


* min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량를 지정한다
* max_depth로 트리의 깊이를 제한하고
* min_samples_split으로 노드를 분할하기 위한 최소 샘플수

In [24]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [25]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [26]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [28]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [29]:
model = DecisionTreeClassifier(max_depth=14, min_impurity_decrease=0.0004, \
                              min_samples_split=12, random_state=42)
model.fit(X_train, y_train)

print('훈련', model.score(X_train, y_train))
print('테스트', model.score(X_test, y_test))

훈련 0.892053107562055
테스트 0.8615384615384616


In [30]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

print('훈련', model.score(X_train, y_train))
print('테스트', model.score(X_test, y_test))

훈련 0.996921300750433
테스트 0.8584615384615385


### 랜덤 서치
* 랜덤서치는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률분포를 전달한다
* scipy의 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 추출해준다
 - 즉, 균등분포에서 샘플링한다 
 - randint는 정수값을, uniform은 실수값을 추출한다
* 임의로 샘플링 하므로 아래와 결과가 다를 수도 있다  

In [31]:
from scipy.stats import uniform, randint

In [32]:
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 7, 6, 7, 4, 8, 0, 0, 8, 7])

* 1000개를 샘플링해서 각 숫자의 개수를 세어본다

In [33]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([109, 108,  93,  93, 100, 108,  95,  92,  96, 106], dtype=int64))

In [34]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.17618235, 0.59150809, 0.97593258, 0.78709444, 0.25405458,
       0.03083526, 0.43932543, 0.55865703, 0.6381422 , 0.64808261])

In [35]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(5, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [36]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F09413E100>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F0942BCF40>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F0942BC9A0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001F0943AC520>},
                   random_state=42)

In [37]:
print(gs.best_params_)

{'max_depth': 45, 'min_impurity_decrease': 0.000404781258158029, 'min_samples_leaf': 6, 'min_samples_split': 9}


In [38]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8693486710594508


In [40]:
dt = gs.best_estimator_

print(dt.score(X_test, y_test))

0.8576923076923076


## 확인문제
* splitter=random 매개변수를 추가하고 다시 훈련
* splitter 매개변수는 기본값은 'best'로 최선의 분할을 찾는다
 - random이면 무작위로 분할 한 다음 가장 좋은 것을 고른다

In [44]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBCABBC6A0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBCAB379D0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBCABBCB80>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BBCABBC7C0>},
                   random_state=42)

In [45]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(X_test, y_test))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
